# Install Dependencies
- ```pip install fastapi```<br>
- ```pip install uvicorn```

# Server
## Import Library

In [ ]:
from py_topping.run_pipeline.api import lazy_API

## Initiate API

In [ ]:
app = lazy_API(   title = 'title'
                , version = 1
                , description= 'testing api'
                , username = 'user' # Default 'user'
                , password = 'password' # Default 'password'
                , api_weak_authen = False # Default False
              )

## Create API
### Post method with Body parameter

In [ ]:
# Create API from Lambda
app.create_post(  function = lambda x : x['a'] + x['b'] # Use Lambda
                , name = 'cal_1'
                , tags = ['cal'] # Default []
                , example = {'a' : 1, 'b' : 2}) # Default {}

# Create API from UDF
def cal_2(x) :
    return x['x'] - x['y']
app.create_post(  function = cal_2
                , name = 'cal_2'
                , tags = ['cal'] # Default []
                , example = {'x' : 10, 'y' : 20}) # Default {}

### Get Method with Query parameter

In [ ]:
def cal_3(x) :
    return x['x'] + x['y']

# Create API from UDF (also work with Lambda)
app.create_get( function = cal_3
              , name = 'cal_3'
              , tags = ['cal_get']) # Default []

## RUN API

In [ ]:
if __name__ == "__main__" : 
    app.run(
             port =8080 # Default 8080
           , host = "0.0.0.0" # Default "0.0.0.0"
           )

---
# Client
## POST If `api_weak_authen` is FALSE

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

r = requests.post('http://localhost:8080/cal_2'
                 , json = {'x' : 10, 'y' : 20}
                 , auth=HTTPBasicAuth('user', 'password')
                 )

print(r, r.content)

## POST If `api_weak_authen` is TRUE

In [ ]:
import requests

r = requests.post('http://localhost:8080/cal_2'
                 , json = {'username' : 'user' , 'password' : 'password'
                           ,'x' : 10, 'y' : 20}
                 )

print(r, r.content)

## GET If `api_weak_authen` is FALSE

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

r = requests.get('http://localhost:8080/cal_3'
                 , params = {'x' : 10, 'y' : 20}
                 , auth=HTTPBasicAuth('user', 'password')
                 )

print(r, r.content)

## GET If `api_weak_authen` is TRUE

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

r = requests.get('http://localhost:8080/cal_3'
                 , params = {'username' : 'user' , 'password' : 'password'
                            ,'x' : 10, 'y' : 20}
                 )

print(r, r.content)